# CSE 291I Final Project
Xuezheng Wang

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import cv2
import glob

warping_field_path = 'outputs/warping_fields/'
frame_path = '../default_input_video/'
temp_frames_path = ''


In [ ]:
if torch.cuda.is_available(): 
    print("Using GPU!")
    dev = "cuda:0" 
else:  
    print("Using CPU!")
    dev = "cpu"  
device = torch.device(dev)  

## Load the frames

In [ ]:
frame_names = sorted(glob.glob(frame_path + "*.png"))
frame_count = len(frame_names)
print('number of frames:', frame_count)

In [ ]:
def readFrame(path):
    frame = torch.tensor(cv2.imread(path))
    frame = torch.stack([frame[:,:,2], frame[:,:,1], frame[:,:,0]], dim=2)
    return frame

In [ ]:
# Now get the dimensions of the frames
sample_frame = readFrame(frame_names[0])
frame_size = sample_frame.shape
print('frame size:', frame_size)

# Show the sample frame
plt.imshow(sample_frame)
plt.show()

# Produce values that will be used later
frame_width = frame_size[1]
frame_height = frame_size[0]

## Optaining Warping Field from [Yu and Ramamorthi]
The field is optained from another program included in the repository. 

In [ ]:
def plot_optical_flow(flow, stride):
    subsampled = np.zeros(np.array(flow.shape[:2]) // stride)
    subsampled = np.reshape([subsampled, subsampled], [subsampled.shape[0], subsampled.shape[1], 2])
#     print(subsampled.shape)

    for i in range(subsampled.shape[0]):
        for j in range(subsampled.shape[1]):
            subsampled[i, j] = flow[i * stride, j * stride]

    plt.quiver(subsampled[:,:,0], subsampled[:,:,1])
    plt.show()

In [ ]:
def homography_to_flow(H, size):
    # Create a meshgrid to transform
    x = np.linspace(0, size[1] - 1, size[1])
    y = np.linspace(0, size[0] - 1, size[0])
    xv, yv = np.meshgrid(x, y)
    
    # Calculate the x, y, and w of each point
    w_new = xv * H[2, 0] + yv * H[2, 1] + H[2, 2]
    x_new = xv * H[0, 0] + yv * H[0, 1] + H[0, 2]
    y_new = xv * H[1, 0] + yv * H[1, 1] + H[1, 2]
    
    x_new = x_new / w_new
    y_new = y_new / w_new
    x_offset = x_new - xv
    y_offset = y_new - yv
    flow = np.dstack([x_offset, y_offset])
    return flow

def resample_flow(flow, new_shape):
    print(new_shape)
    print(flow.shape)
    flow = torch.from_numpy(flow)
    flow = flow.permute(2, 0, 1).unsqueeze(0)
    print(flow.shape)
    new_flow = F.interpolate(flow, new_shape, mode='bilinear')
    print(new_flow.shape)
    return_val = new_flow[0].permute(1, 2, 0)
    print(return_val.shape)
    return return_val
    

In [ ]:
# Find the path to all the H_inv and warping fields
path = 'outputs/warping_fields/'

# 
# Load the warping field of a given frame idx
#
def load_warping_field(index):
    # Make sure we have the index
    assert index >= 0 and index < frame_count
    
    # Initialize the variables
    H_inv = None
    flow = None
    
    # Special treatment for frame 0
    flow_shape = np.array([576, 960, 2]) # As generated by other code
    if index == 0:
        H_inv = np.eye(3)
        flow = np.zeros(flow_shape) # Why are we using 448 + 2 * 64?
    else:
        H_inv_path = path + str(index).zfill(5) + '_H_inv.npy'
        H_inv = np.load(H_inv_path)

        flow_path = path + str(index).zfill(5) + '.npy'
        flow = np.load(flow_path)
        
        # Convert optical flow space from [-1, 1] to [0, width/height]
        flow[:, :, 0] *= float(flow_shape[0]) / 2
        flow[:, :, 1] *= float(flow_shape[1]) / 2
        
    # Combine the optical flow and the homography
    H_flow = homography_to_flow(H_inv, flow_shape)
    combined_flow = H_flow + flow
        
    # Resample the optical flow to frame space
    # There's 64 paddings around the edge
    padding = 64
    frame_flow = resample_flow(combined_flow[padding:-padding, padding:-padding], frame_size[:2])
    plot_optical_flow(combined_flow, 30)
    plot_optical_flow(frame_flow, 30)
    
    return frame_flow


flow = load_warping_field(3)

In [ ]:
iterator = sorted(glob.glob(warping_field_path + '*.npy'))
for path in iterator:
    print(path)



## Use RAFT for optical flow